# 데이터 검색

이 노트북에서는 Azure AI Search의 다양한 검색 방법(키워드, 벡터, 하이브리드)을 수행합니다.

## 0. 필요한 라이브러리 설치

In [1]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents openai

Note: you may need to restart the kernel to use updated packages.


## 1. 환경 변수 설정

In [2]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Embedding Model: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")

Search Endpoint: https://ai-search-commercial-1023.search.windows.net
Search Index: products-index
OpenAI Endpoint: https://open-ai-seoulcentral-1023.openai.azure.com/
Embedding Model: text-embedding-3-large


## 2. 클라이언트 초기화

In [3]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from IPython.display import display, Image, HTML
import os, shutil

##############################################
# 2-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))

##############################################
# 2-1. Azure 인증 및 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

openai_client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

##############################################
# 2-2. 임베딩 생성 함수
##############################################
def get_embedding(text):
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT
    )
    return response.data[0].embedding

print("클라이언트 초기화 완료")

클라이언트 초기화 완료


# 3. Analyzer 테스트

In [4]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AnalyzeTextOptions

##############################################
# 3. Analyzer 테스트
##############################################

# 클라이언트 생성
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    credential=credential
)

# Sample Query
query = "튼튼하게 만들어진 백팩으로, 컴퓨터를 넣을 수 있어야함."

# ko.microsoft
opts_ms = AnalyzeTextOptions(text=query, analyzer_name="ko.microsoft")
result_ms = index_client.analyze_text(AZURE_SEARCH_INDEX_NAME, opts_ms)
print("[ko.microsoft]", [t.token for t in result_ms.tokens])

# ko.lucene
opts_lu = AnalyzeTextOptions(text=query, analyzer_name="ko.lucene")
result_lu = index_client.analyze_text(AZURE_SEARCH_INDEX_NAME, opts_lu)
print("[ko.lucene]", [t.token for t in result_lu.tokens])

[ko.microsoft] ['튼튼하게', '튼튼', '만들어진', '백팩으로', '백팩', '컴퓨터를', '컴퓨터', '넣을', '수', '있어야함']
[ko.lucene] ['튼튼', '튼하', '하게', '만들', '들어', '어진', '백팩', '팩으', '으로', '컴퓨', '퓨터', '터를', '넣을', '수', '있어', '어야', '야함']


## 4. 키워드 검색 (Full-text Search)

In [5]:
##############################################
# 4. 키워드 검색
##############################################
search_query = "튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함."

results = search_client.search(
    search_text=search_query,
    top=5,
    query_type="full",
    search_fields=["name", "description"],
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[키워드 검색 결과]")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   설명: {result['description']}")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

검색어: '튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함.'

[키워드 검색 결과]

1. 린넨 와이드 팬츠 (내츄럴룩)
   설명: 통풍이 잘되는 린넨 소재의 와이드 팬츠입니다. 여름철 시원하고 편안한 착용감을 제공합니다.



2. 클래식 화이트 티셔츠 (베이직코튼)
   설명: 100% 순면으로 제작된 기본 화이트 티셔츠입니다. 데일리룩으로 완벽하며 편안한 착용감을 제공합니다.



3. 오버핏 후드티 (스트릿웨어)
   설명: 트렌디한 오버핏 후드티로 편안한 착용감과 스타일을 동시에 제공합니다. 기모 안감으로 따뜻합니다.



4. 플로럴 원피스 (로맨틱스타일)
   설명: 봄 여름 시즌에 어울리는 플로럴 패턴의 원피스입니다. 여성스러운 디자인과 편안한 착용감이 특징입니다.



5. 레더 첼시 부츠 (프리미엄슈즈)
   설명: 이탈리아산 천연 가죽으로 제작된 첼시 부츠입니다. 세련된 디자인과 편안한 착용감이 특징입니다.


## 5. 벡터 검색 (Vector Search)

In [6]:
from azure.search.documents.models import VectorizedQuery

##############################################
# 5. 벡터 검색
##############################################
search_query = "튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함."

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["id", "name", "brand", "price", "description", "imageUrl", "descriptionVector"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[벡터 검색 결과]")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result['@search.score']
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   유사도 점수: {score:.4f}")
    print(f"   설명: {result['description']}")
    print(f"   설명 벡터 (처음 5개 값): {result['descriptionVector'][:5]}...")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

검색어: '튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함.'

[벡터 검색 결과]

1. 캔버스 백팩 (어반트래블)
   유사도 점수: 0.6931
   설명: 내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.
   설명 벡터 (처음 5개 값): [-0.021477036, -0.019682806, 0.00034162213, 0.013945569, -0.009046355]...



2. 숄더백 (어바에센셜)
   유사도 점수: 0.6770
   설명: 심플한 디자인의 가죽 숄더백입니다. 수납공간이 넉넉하고 일상 사용에 편리합니다.
   설명 벡터 (처음 5개 값): [-0.0058374587, 0.0007554125, -0.0007547528, -0.0017311812, -0.006713605]...



3. 가죽 크로스백 (럭셔리레더)
   유사도 점수: 0.6142
   설명: 고급 천연 가죽으로 제작된 크로스백입니다. 심플한 디자인으로 어떤 스타일과도 잘 어울립니다.
   설명 벡터 (처음 5개 값): [-0.003048197, -0.01604432, -0.00580529, -0.0038431354, 0.0026157394]...



4. 코튼 조거 팬츠 (컴포트웨어)
   유사도 점수: 0.5977
   설명: 부드러운 코튼 소재의 조거 팬츠입니다. 홈웨어로도, 가볍운 외출복으로도 활용 가능합니다.
   설명 벡터 (처음 5개 값): [-0.023708804, 0.015363218, -0.0064832135, 0.007913731, 0.03066166]...



5. 오버핏 후드티 (스트릿웨어)
   유사도 점수: 0.5912
   설명: 트렌디한 오버핏 후드티로 편안한 착용감과 스타일을 동시에 제공합니다. 기모 안감으로 따뜻합니다.
   설명 벡터 (처음 5개 값): [-0.047118954, -0.013244415, -0.006840351, -0.0017555342, -0.0052172607]...


## 6. 하이브리드 검색 (Hybrid Search)

In [7]:
##############################################
# 6. 하이브리드 검색 (키워드 + 벡터)
##############################################
search_query = "튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함."

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=5,
    select=["id", "name", "brand", "price", "description", "imageUrl", "descriptionVector"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[하이브리드 검색 결과]")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result['@search.score']
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   통합 점수: {score:.4f}")
    print(f"   설명: {result['description']}")
    print(f"   설명 벡터 (처음 5개 값): {result['descriptionVector'][:5]}...")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

검색어: '튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함.'

[하이브리드 검색 결과]

1. 오버핏 후드티 (스트릿웨어)
   통합 점수: 0.0318
   설명: 트렌디한 오버핏 후드티로 편안한 착용감과 스타일을 동시에 제공합니다. 기모 안감으로 따뜻합니다.
   설명 벡터 (처음 5개 값): [-0.047118954, -0.013244415, -0.006840351, -0.0017555342, -0.0052172607]...



2. 캔버스 백팩 (어반트래블)
   통합 점수: 0.0167
   설명: 내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.
   설명 벡터 (처음 5개 값): [-0.021477036, -0.019682806, 0.00034162213, 0.013945569, -0.009046355]...



3. 린넨 와이드 팬츠 (내츄럴룩)
   통합 점수: 0.0167
   설명: 통풍이 잘되는 린넨 소재의 와이드 팬츠입니다. 여름철 시원하고 편안한 착용감을 제공합니다.
   설명 벡터 (처음 5개 값): [-0.016942248, -0.016199958, 0.007177133, 0.0054417816, 0.009052917]...



4. 클래식 화이트 티셔츠 (베이직코튼)
   통합 점수: 0.0164
   설명: 100% 순면으로 제작된 기본 화이트 티셔츠입니다. 데일리룩으로 완벽하며 편안한 착용감을 제공합니다.
   설명 벡터 (처음 5개 값): [-0.0012417478, -0.049078938, 0.004632106, 0.00014915045, -0.0010370177]...



5. 숄더백 (어바에센셜)
   통합 점수: 0.0164
   설명: 심플한 디자인의 가죽 숄더백입니다. 수납공간이 넉넉하고 일상 사용에 편리합니다.
   설명 벡터 (처음 5개 값): [-0.0058374587, 0.0007554125, -0.0007547528, -0.0017311812, -0.006713605]...


## 7. 하이브리드 검색 (Hybrid Search) + Semantic Rank

### 7.1. Semantic Rank  추가

In [8]:
##############################################
# 7. Semantic Rank 추가
##############################################


# Add semantic configuration to hotels-sample-index and display updated index details
from azure.search.documents.indexes.models import (
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)


# Get the existing index
existing_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

# Create a new semantic configuration
new_semantic_config = SemanticConfiguration(
    name="product-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="name"), 
        content_fields=[SemanticField(field_name="description")]
    )
)

# Add semantic configuration to the index
if existing_index.semantic_search is None:
    existing_index.semantic_search = SemanticSearch(configurations=[new_semantic_config])
else:
    # Check if configuration already exists
    config_exists = any(config.name == "product-semantic-config" 
                        for config in existing_index.semantic_search.configurations)
    if not config_exists:
        existing_index.semantic_search.configurations.append(new_semantic_config)

# Update the index
result = index_client.create_or_update_index(existing_index)

# Get the updated index and display detailed information
updated_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

print("Semantic configurations:")
print("-" * 40)
if updated_index.semantic_search and updated_index.semantic_search.configurations:
    for config in updated_index.semantic_search.configurations:
        print(f"  Configuration: {config.name}")
        if config.prioritized_fields.title_field:
            print(f"    Title field: {config.prioritized_fields.title_field.field_name}")
        if config.prioritized_fields.keywords_fields:
            keywords = [kf.field_name for kf in config.prioritized_fields.keywords_fields]
            print(f"    Keywords fields: {', '.join(keywords)}")
        if config.prioritized_fields.content_fields:
            content = [cf.field_name for cf in config.prioritized_fields.content_fields]
            print(f"    Content fields: {', '.join(content)}")
        print()
else:
    print("  No semantic configurations found")

print("✅ Semantic configuration successfully added!")


Semantic configurations:
----------------------------------------
  Configuration: product-semantic-config
    Title field: name
    Content fields: description

✅ Semantic configuration successfully added!


### 7.2. Run Query

In [9]:
from azure.search.documents.models import VectorizedQuery

##############################################
# 7.2. 하이브리드 검색 (Hybrid Search) + Semantic Rank 쿼리
##############################################

search_query = "튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함."

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector",
    kind="vector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    query_type="semantic",
    semantic_configuration_name="product-semantic-config",
    top=5,
    select=["id", "name", "brand", "price", "description", "imageUrl", "descriptionVector"],
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[하이브리드 + Semantic Ranker 검색 결과]")
print(f"{'='*60}")

for idx, result in enumerate(results, 1):
    score = result.get("@search.score", "N/A")
    reranker_score = result.get("@search.reranker_score", "N/A")
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   검색 점수: {score}")
    print(f"   Semantic Re-ranker Score: {reranker_score}")
    print(f"   설명: {result['description']}")
    print(f"   벡터 (처음 5개 값): {result['descriptionVector'][:5]}...")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))


검색어: '튼튼하고 편안한 가방으로, 컴퓨터를 넣을 수 있어야함.'

[하이브리드 + Semantic Ranker 검색 결과]
1. 캔버스 백팩 (어반트래블)
   검색 점수: 0.01666666753590107
   Semantic Re-ranker Score: 2.144556999206543
   설명: 내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.
   벡터 (처음 5개 값): [-0.021477036, -0.019682806, 0.00034162213, 0.013945569, -0.009046355]...


2. 린넨 와이드 팬츠 (내츄럴룩)
   검색 점수: 0.01666666753590107
   Semantic Re-ranker Score: 2.142230749130249
   설명: 통풍이 잘되는 린넨 소재의 와이드 팬츠입니다. 여름철 시원하고 편안한 착용감을 제공합니다.
   벡터 (처음 5개 값): [-0.016942248, -0.016199958, 0.007177133, 0.0054417816, 0.009052917]...


3. 오버핏 후드티 (스트릿웨어)
   검색 점수: 0.0317540317773819
   Semantic Re-ranker Score: 2.0553858280181885
   설명: 트렌디한 오버핏 후드티로 편안한 착용감과 스타일을 동시에 제공합니다. 기모 안감으로 따뜻합니다.
   벡터 (처음 5개 값): [-0.047118954, -0.013244415, -0.006840351, -0.0017555342, -0.0052172607]...


4. 레더 첼시 부츠 (프리미엄슈즈)
   검색 점수: 0.015625
   Semantic Re-ranker Score: 2.0356242656707764
   설명: 이탈리아산 천연 가죽으로 제작된 첼시 부츠입니다. 세련된 디자인과 편안한 착용감이 특징입니다.
   벡터 (처음 5개 값): [-0.04274861, -0.014036403, -0.011468644, 0.017334906, 0.005937307]...


5. 플로럴 원피스 (로맨틱스타일)
   검색 점수: 0.01587301678955555
   Semantic Re-ranker Score: 1.800079345703125
   설명: 봄 여름 시즌에 어울리는 플로럴 패턴의 원피스입니다. 여성스러운 디자인과 편안한 착용감이 특징입니다.
   벡터 (처음 5개 값): [-0.04576321, 0.0047428985, -3.7828027e-05, -0.0016416694, -0.003103907]...


### 7.3. Semantic Configuration 제거

In [10]:
##############################################
# 7.3. Semantic Configuration 제거
##############################################

DELETE_SEMANTIC_CONFIG = False

if DELETE_SEMANTIC_CONFIG:
    # 기존 인덱스 가져오기
    existing_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

    # product-semantic-config 제거
    if existing_index.semantic_search and existing_index.semantic_search.configurations:
        print("\n⚠️  'product-semantic-config' 제거 중...")
        
        # 해당 config 제거
        original_count = len(existing_index.semantic_search.configurations)
        existing_index.semantic_search.configurations = [
            config for config in existing_index.semantic_search.configurations 
            if config.name != "product-semantic-config"
        ]
        removed_count = original_count - len(existing_index.semantic_search.configurations)
        
        # configurations가 비어있으면 빈 리스트로 유지 (None이 아닌)
        # Azure Search는 빈 리스트를 허용함
        
        try:
            # 인덱스 업데이트
            index_client.create_or_update_index(existing_index)
            print(f"✅ {removed_count}개의 Configuration 제거 완료")
        except Exception as e:
            print(f"❌ 제거 실패: {e}")
            raise
    else:
        print("\nℹ️  제거할 Semantic Configuration이 없습니다.")